## **1. 모델 이해하기**

우리가 만들 모델의 아키텍처를 이해해봅시다. 이번 모델의 아키텍처는 총 5개의 층으로 구성됩니다. 앞서 배운 챕터에서 1번 레이어와 2번 레이어는 동일하되, 새로운 합성곱층과 전결합층을 추가했습니다.

> 1번 레이어 : 합성곱층(Convolutional layer)  
>합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
>맥스풀링(kernel_size=2, stride=2))
>
> 2번 레이어 : 합성곱층(Convolutional layer)  
>합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
>맥스풀링(kernel_size=2, stride=2))
>
> 3번 레이어 : 합성곱층(Convolutional layer)  
>합성곱(in_channel = 64, out_channel = 128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
>맥스풀링(kernel_size=2, stride=2, padding=1))
>
> 4번 레이어 : 전결합층(Fully-Connected layer)  
>특성맵을 펼친다. # batch_size × 4 × 4 × 128 → batch_size × 2048
>전결합층(뉴런 625개) + 활성화 함수 ReLU
>
> 5번 레이어 : 전결합층(Fully-Connected layer)  
>전결합층(뉴런 10개) + 활성화 함수 Softmax

## **2. 깊은 CNN으로 MNIST 분류하기**

이번 모델은 저번 챕터 모델에서 층이 추가되는 것 외에는 동일합니다.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                        train=True, # True를 지정하면 훈련 데이터로 다운로드
                        transform=transforms.ToTensor(), # 텐서로 변환
                        download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                        train=False, # False를 지정하면 테스트 데이터로 다운로드
                        transform=transforms.ToTensor(), # 텐서로 변환
                        download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [6]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [7]:
# CNN 모델 정의
model = CNN().to(device)

# 비용 함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [9]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.19166024
[Epoch:    2] cost = 0.0549238361
[Epoch:    3] cost = 0.0394187085
[Epoch:    4] cost = 0.0302530304
[Epoch:    5] cost = 0.0255598389
[Epoch:    6] cost = 0.0210455712
[Epoch:    7] cost = 0.0177619569
[Epoch:    8] cost = 0.0159254745
[Epoch:    9] cost = 0.0135003021
[Epoch:   10] cost = 0.0124604004
[Epoch:   11] cost = 0.012707023
[Epoch:   12] cost = 0.00989794638
[Epoch:   13] cost = 0.00910468586
[Epoch:   14] cost = 0.0082603246
[Epoch:   15] cost = 0.00962453522


In [10]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\delphinus\anaconda3\envs\nlp\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9847000241279602


층을 더 깊게 쌓았는데 오히려 정확도가 줄어드는 것을 볼 수 있습니다. 결국 층을 깊게 쌓는 것도 중요하지만, 꼭 깊게 쌓는 것이 정확도를 올려주지는 않으며 효율적으로 쌓는 것도 중요하다는 의미입니다.